In [ ]:
import os
import json
import boto3
from tqdm import tqdm

def download_and_arrange_data():
    s3_client = boto3.client('s3')

    with open('file_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items():
        print(f"Downloading Images with {k} objects")
        directory=os.path.join('train_data', k)
        if not os.path.exists(directory):
            os.makedirs(directory)
        for file_path in tqdm(v):
            file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
            s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name),
                             os.path.join(directory, file_name))

download_and_arrange_data()

## Create Test Dataset by using sample 100 images from each subfolder

In [10]:
import os
import random
import shutil

# Paths
source_parent_folder = "./train_data"
test_parent_folder = "./dataset/test"
num_images_to_sample = 100

# List of subfolder names
subfolder_names = os.listdir(source_parent_folder)
print(subfolder_names)

# Loop through each subfolder
for subfolder_name in subfolder_names:
    source_subfolder = os.path.join(source_parent_folder, subfolder_name)
    test_subfolder = os.path.join(test_parent_folder, subfolder_name)

    # Create target subfolder in 'test' folder if it doesn't exist
    os.makedirs(test_subfolder, exist_ok=True)

    # List all files in the source subfolder
    all_files = os.listdir(source_subfolder)
    image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Randomly sample 100 images
    selected_images = random.sample(image_files, num_images_to_sample)

    # Move selected images to the test subfolder
    for image_name in selected_images:
        source_path = os.path.join(source_subfolder, image_name)
        target_path = os.path.join(test_subfolder, image_name)
        shutil.move(source_path, target_path)

    # # Remove moved images from the source subfolder
    # for image_name in selected_images:
    #     source_path = os.path.join(source_subfolder, image_name)
    #     os.remove(source_path)

print("Images moved to 'test' folders and removed from source folders successfully.")

['4', '2', '1', '3', '5']
Images moved to 'test' folders and removed from source folders successfully.


## Create Train and Val

In [11]:
!pip install split-folders

In [12]:
import splitfolders
splitfolders.ratio('./train_data', output="./dataset", seed=1337, ratio=(.8, 0.2))

Copying files: 9941 files [00:02, 4364.33 files/s]


In [30]:
# !aws s3 cp dataset s3://dataset-capstone/ --recursive